In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [25]:
# Hyper Parameters
EPOCHS = 4
batch_size = 4
learning_rate = 0.001

In [26]:
# Dataset has PIL-Images of range [0,1]
# We Trasnform them to tensors of nromalized range [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [27]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified


In [28]:
# Implment Conv net

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)  # # input_layer : 3 input channels , 6 output channels , 5 kernel_size ( flitering)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5) # input here must be same number as output channel in conv1
        
        self.linear1 = nn.Linear(16*5*5,120) # 16*5*5 = 
        self.linear2 = nn.Linear(120,84)
        self.linear3 = nn.Linear(84,10) # output must be 10
        
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5) # Flattening it
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

model = ConvNet().to(device)

In [29]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [31]:
# Training Loop

n_total_steps = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')


Epoch [1/4], Step [2000/12500], Loss: 2.3163
Epoch [1/4], Step [4000/12500], Loss: 2.3454
Epoch [1/4], Step [6000/12500], Loss: 2.3015
Epoch [1/4], Step [8000/12500], Loss: 2.0420
Epoch [1/4], Step [10000/12500], Loss: 2.8965
Epoch [1/4], Step [12000/12500], Loss: 2.1090
Epoch [2/4], Step [2000/12500], Loss: 2.5650
Epoch [2/4], Step [4000/12500], Loss: 1.8504
Epoch [2/4], Step [6000/12500], Loss: 2.4940
Epoch [2/4], Step [8000/12500], Loss: 1.5247
Epoch [2/4], Step [10000/12500], Loss: 1.5881
Epoch [2/4], Step [12000/12500], Loss: 2.6673
Epoch [3/4], Step [2000/12500], Loss: 1.5817
Epoch [3/4], Step [4000/12500], Loss: 1.6174
Epoch [3/4], Step [6000/12500], Loss: 2.3095
Epoch [3/4], Step [8000/12500], Loss: 1.4741
Epoch [3/4], Step [10000/12500], Loss: 1.3350
Epoch [3/4], Step [12000/12500], Loss: 1.6069
Epoch [4/4], Step [2000/12500], Loss: 1.7630
Epoch [4/4], Step [4000/12500], Loss: 0.8575
Epoch [4/4], Step [6000/12500], Loss: 1.4452
Epoch [4/4], Step [8000/12500], Loss: 0.6463
Epoc

In [32]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 47.73 %
Accuracy of plane: 52.4 %
Accuracy of car: 65.9 %
Accuracy of bird: 30.8 %
Accuracy of cat: 30.7 %
Accuracy of deer: 20.4 %
Accuracy of dog: 44.6 %
Accuracy of frog: 62.9 %
Accuracy of horse: 50.3 %
Accuracy of ship: 61.3 %
Accuracy of truck: 58.0 %
